### [Event data] Import events

In [101]:
import pandas as pd

us_events = pd.read_csv('../data/stocks/tesla_nasdag_event_study.csv')
ger_events = pd.read_csv('../data/stocks/tesla_xetra_event_study.csv')

us_events['event_time'] = pd.to_datetime(us_events['event_time'],  utc = True).dt.tz_convert(
    tz="US/Eastern"
)
ger_events['event_time'] = pd.to_datetime(ger_events['event_time'],  utc = True).dt.tz_convert(
    tz="US/Eastern"
)

us_events.drop(columns=['Unnamed: 0'], inplace=True)
ger_events.drop(columns=['Unnamed: 0'], inplace=True)

merged_events = us_events.merge(ger_events, on=['event_time', 'min_offset'], how='inner', suffixes=('_US', '_GER')).fillna(0)

In [102]:
merged_events['min_offset'] = merged_events['min_offset'].astype('float')

merged_events['car_us'] = merged_events.groupby('event_time')['abnormal_return_US'].cumsum()
merged_events['car_ger'] = merged_events.groupby('event_time')['abnormal_return_GER'].cumsum()

In [103]:
merged_events

,event_time,min_offset,log_return_US,expected_return_US,abnormal_return_US,tweet,log_return_GER,expected_return_GER,abnormal_return_GER,car_us,car_ger
0,2024-02-01 18:49:00-05:00,-300.0,0.000845,0.000000,0.000000,worth readingnew house speaker mike johnson de...,0.0,0.000000,0.000000,0.000000,0.000000
1,2024-02-01 18:49:00-05:00,-299.0,0.000059,0.000000,0.000000,worth readingnew house speaker mike johnson de...,0.0,0.000000,0.000000,0.000000,0.000000
2,2024-02-01 18:49:00-05:00,-298.0,-0.000213,0.000000,0.000000,worth readingnew house speaker mike johnson de...,0.0,0.000000,0.000000,0.000000,0.000000
3,2024-02-01 18:49:00-05:00,-297.0,-0.000213,0.000000,0.000000,worth readingnew house speaker mike johnson de...,0.0,0.000000,0.000000,0.000000,0.000000
4,2024-02-01 18:49:00-05:00,-296.0,0.000372,0.000000,0.000000,worth readingnew house speaker mike johnson de...,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
63487,2025-04-24 10:49:00-04:00,176.0,-0.000981,0.000097,-0.001078,scam centurya new legal letter aimed openai la...,0.0,0.000115,-0.000115,-0.017455,-0.025644
63488,2025-04-24 10:49:00-04:00,177.0,0.001608,0.000097,0.001511,scam centurya new legal letter aimed openai la...,0.0,0.000115,-0.000115,-0.015945,-0.025759
63489,2025-04-24 10:49:00-04:00,178.0,0.000627,0.000097,0.000530,scam centurya new legal letter aimed openai la...,0.0,0.000115,-0.000115,-0.015415,-0.025873
63490,2025-04-24 10:49:00-04:00,179.0,-0.001176,0.000097,-0.001273,scam centurya new legal letter aimed openai la...,0.0,0.000115,-0.000115,-0.016688,-0.025988


In [104]:
print(len(us_events['event_time'].unique()))
print(len(ger_events['event_time'].unique()))
print(len(merged_events['event_time'].unique()))

196
188
132


In [105]:
df_fx = pd.read_csv('../data/stocks/usd_eur_230501-250501.csv')
df_fx = df_fx.rename(columns={'Unnamed: 0': 'event_date'})

In [106]:
# # Step 1: Extract just the date from event_time (remove time component)
# merged_events['event_date'] = merged_events['event_time'].dt.date

# # Step 2: Make sure df_fx['Date'] is also in date format
# df_fx['event_date'] = pd.to_datetime(df_fx['event_date']).dt.date

# # Step 3: Merge on the date column
# merged_events = merged_events.merge(df_fx, on='event_date', how='left')
# merged_events

### t-test

Let 
x = 
abnormal_return_US, 
y =
abnormal_return_GER


Null Hypothesis 
𝐻0: 
𝜇
𝑋
−
𝜇
𝑌
≤
0
(US abnormal return is not greater than Germany)

In [107]:
# test_ar_results_copy = test_ar_results[(test_ar_results['p_val_one_sided'] < 0.05) &
#                                   (test_ar_results['start_min'] < test_ar_results['end_min'])]
# test_ar_results_copy

In [108]:
t_stat, p_val_left_sided = ttest_rel(merged_events['car_us'], merged_events['car_ger'], alternative='two-sided')
print(f"Paired t-test — T-statistic: {t_stat:.4f}, p-value: {p_val_left_sided:.4f}")

Paired t-test — T-statistic: -9.6960, p-value: 0.0000


In [109]:
from scipy.stats import ttest_rel

for min in merged_events['min_offset'].unique():
    if min < 0:
        continue
    AAR_us = merged_events[merged_events['min_offset'] == min]['car_us']
    AAR_ger = merged_events[merged_events['min_offset'] == min]['car_ger']

    t_stat, p_val_left_sided = ttest_rel(AAR_ger, AAR_us, alternative='less')
    if p_val_left_sided < 0.1:
        print('for minute: ' ,min, ' ',t_stat, p_val_left_sided)

In [117]:
from numpy import exp

# Define the windows
windows = [(0,2), (0, 5), (0,10), (0,30), (0,60), (0,120), (0,180)]

# Prepare results
results = []

for start_min, end_min in windows:
    # Filter relevant window
    df_window = merged_events[(merged_events["min_offset"] >= start_min) & (merged_events["min_offset"] <= end_min)]
    print(len(df_window))
    
    # Compute CARs per event
    #car_per_event_ger = df_window.groupby("event_time")["abnormal_return_GER"].sum()
    car_per_event_ger = abs(df_window[df_window['min_offset']==end_min]["car_ger"])

    #car_per_event_us = df_window.groupby("event_time")["abnormal_return_US"].sum()
    car_per_event_us = abs(df_window[df_window['min_offset']==end_min]["car_us"])

    # Perform one-sample t-test against 0
    t_stat_1, p_val_two_sided = ttest_rel(car_per_event_ger, car_per_event_us, alternative='two-sided')
    t_stat_2, p_val_left_sided = ttest_rel(car_per_event_ger, car_per_event_us, alternative='less')
    
    # Store results
    results.append({
        "Window": f"[{start_min},{end_min}]",
        "T-Stat": t_stat_2,
        'CAR_US': car_per_event_us.mean(),
        'CAR_GER': car_per_event_ger.mean(),
        "P-Value (Left-Sided)": p_val_left_sided,
        "P-Value (Two-Sided)": p_val_two_sided
    })

test_car_results = pd.DataFrame(results)
test_car_results = test_car_results[["Window", "T-Stat",'CAR_US', 'CAR_GER', "P-Value (Left-Sided)", "P-Value (Two-Sided)"]]
test_car_results

396
792
1452
4092
8052
15972
23892


,Window,T-Stat,CAR_US,CAR_GER,P-Value (Left-Sided),P-Value (Two-Sided)
0,"[0,2]",-4.661643,0.001419,0.000665,0.000004,0.000008
1,"[0,5]",-2.844025,0.002089,0.001431,0.002585,0.005170
2,"[0,10]",-3.165400,0.002887,0.001951,0.000963,0.001926
3,"[0,30]",-1.023304,0.006028,0.005569,0.154025,0.308050
4,"[0,60]",-1.364756,0.008526,0.007885,0.087335,0.174669
5,"[0,120]",-1.261915,0.011946,0.011165,0.104611,0.209222
6,"[0,180]",-2.443057,0.016724,0.014790,0.007948,0.015895


In [111]:
from scipy.stats import ttest_rel

# Define the windows
windows = [(0,2), (0, 5), (0,10), (0,30), (0,60), (0,120), (0,180)]

# Prepare results
results = []

for start_min, end_min in windows:
    # Filter relevant window
    df_window = merged_events[(merged_events["min_offset"] >= start_min) & (merged_events["min_offset"] <= end_min)]
    print(len(df_window))
    
    # Compute CARs per event
    car_per_event_ger = df_window.groupby("event_time")["abnormal_return_GER"].sum()
    car_per_event_us = df_window.groupby("event_time")["abnormal_return_US"].sum()

    # Perform one-sample t-test against 0
    t_stat, p_val_left_sided = ttest_rel(car_per_event_ger, car_per_event_us, alternative='less')
    
    # Store results
    results.append({
        "Window": f"[{start_min},{end_min}]",
        "T-Stat": t_stat,
        "P-Value (Left-Sided)": p_val_left_sided
    })

test_car_results = pd.DataFrame(results)
test_car_results = test_car_results[["Window", "T-Stat", "P-Value (Left-Sided)"]]
test_car_results

396
792
1452
4092
8052
15972
23892


,Window,T-Stat,P-Value (Left-Sided)
0,"[0,2]",0.475919,0.682538
1,"[0,5]",0.899302,0.814929
2,"[0,10]",-0.102734,0.459166
3,"[0,30]",1.187914,0.881492
4,"[0,60]",1.275392,0.897786
5,"[0,120]",1.374017,0.914109
6,"[0,180]",0.247699,0.597622


In [112]:
no_of_events = []
print(len(merged_events['event_time'].unique()))
for event in merged_events['event_time'].unique():
    #print(event)
    df_window = merged_events[merged_events['event_time'] == event]
    
    if len(df_window) > 0:
        # Compute CARs per event
        car_per_event_ger = df_window[df_window['min_offset'].between(1, 180)]["car_ger"]
        car_per_event_us = df_window[df_window['min_offset'].between(1, 180)]["car_us"]

        # Perform one-sample t-test against 0
        t_stat, p_val_left_sided = ttest_rel( car_per_event_us,  car_per_event_ger ,alternative='greater')

        if p_val_left_sided < 0.1:
            no_of_events.append(event)
            print(f"Event: {event}, T-Stat: {t_stat}, P-Value (Left-Sided): {p_val_left_sided}")
print(f"{no_of_events} Number of events with significant difference: {len(no_of_events)}")

132
Event: 2024-02-15 14:12:00-05:00, T-Stat: 19.145305843868005, P-Value (Left-Sided): 1.7507967504443094e-45
Event: 2024-02-22 18:07:00-05:00, T-Stat: 24.56014613897221, P-Value (Left-Sided): 1.614207855110532e-59


Event: 2024-02-26 10:27:00-05:00, T-Stat: 15.897727986358934, P-Value (Left-Sided): 2.3260677302528753e-36
Event: 2024-02-27 13:40:00-05:00, T-Stat: 32.17335447175322, P-Value (Left-Sided): 1.2499094872486837e-76
Event: 2024-03-19 10:57:00-04:00, T-Stat: 23.810580489736985, P-Value (Left-Sided): 1.1372559822736289e-57
Event: 2024-03-21 09:14:00-04:00, T-Stat: 5.755237107537505, P-Value (Left-Sided): 1.8440293488192405e-08
Event: 2024-03-26 14:14:00-04:00, T-Stat: 12.368666434217836, P-Value (Left-Sided): 4.2630485876461127e-26
Event: 2024-04-05 11:57:00-04:00, T-Stat: 1.7029224046731666, P-Value (Left-Sided): 0.04515914584326142
Event: 2024-04-11 11:36:00-04:00, T-Stat: 21.660720655629508, P-Value (Left-Sided): 3.359868447570611e-52
Event: 2024-04-15 16:35:00-04:00, T-Stat: 30.499827627350765, P-Value (Left-Sided): 4.094603448759701e-73
Event: 2024-04-16 11:40:00-04:00, T-Stat: 13.3300464572271, P-Value (Left-Sided): 6.654243839596693e-29
Event: 2024-04-19 14:11:00-04:00, T-Stat: 12.76

In [113]:
for event in no_of_events:
    print(event)

2024-02-15 14:12:00-05:00
2024-02-22 18:07:00-05:00
2024-02-26 10:27:00-05:00
2024-02-27 13:40:00-05:00
2024-03-19 10:57:00-04:00
2024-03-21 09:14:00-04:00
2024-03-26 14:14:00-04:00
2024-04-05 11:57:00-04:00
2024-04-11 11:36:00-04:00
2024-04-15 16:35:00-04:00
2024-04-16 11:40:00-04:00
2024-04-19 14:11:00-04:00
2024-04-22 12:08:00-04:00
2024-04-23 13:19:00-04:00
2024-04-26 16:03:00-04:00
2024-05-06 13:14:00-04:00
2024-05-13 16:36:00-04:00
2024-05-20 18:54:00-04:00
2024-07-08 13:18:00-04:00
2024-07-12 12:28:00-04:00
2024-07-15 12:21:00-04:00
2024-07-16 15:14:00-04:00
2024-07-19 13:42:00-04:00
2024-08-09 15:02:00-04:00
2024-08-12 15:15:00-04:00
2024-09-02 11:54:00-04:00
2024-09-09 10:32:00-04:00
2024-09-11 10:31:00-04:00
2024-09-12 17:42:00-04:00
2024-09-23 18:06:00-04:00
2024-10-04 10:32:00-04:00
2024-10-21 10:47:00-04:00
2024-10-30 12:05:00-04:00
2024-11-05 12:10:00-05:00
2024-11-19 09:02:00-05:00
2024-11-25 11:03:00-05:00
2024-12-06 12:35:00-05:00
2024-12-10 09:08:00-05:00
2024-12-11 1